### Importar las librerías

In [9]:
import os 
import re 
from PyPDF2 import PdfReader
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from transformers import AutoTokenizer
from langchain_huggingface import HuggingFaceEmbeddings 
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser 
from langchain_ollama.llms import OllamaLLM 
from langchain_core.runnables import RunnablePassthrough

### Extraer datos del pdf

In [10]:
# abrir pdf
with open("clima.pdf", "rb") as file:
    lector = PdfReader(file)
    textos_pdf = [pagina.extract_text() for pagina in lector.pages]

# previsualizar algunos textos
for texto in textos_pdf[:5]:
    print(texto)

Clima
en
A
Coruña
esta
Semana
Lunes,
18
de
diciembre:
●
Temperatura
máxima:
16°C
●
Temperatura
mínima:
10°C
●
Probabilidad
de
lluvia:
60%
●
Viento:
13
km/h
Martes,
19
de
diciembre:
●
Temperatura
máxima:
16°C
●
Temperatura
mínima:
9°C
●
Probabilidad
de
lluvia:
90%
●
Viento:
33
km/h
Miércoles,
20
de
diciembre:
●
Temperatura
máxima:
18°C
●
Temperatura
mínima:
12°C
●
Probabilidad
de
lluvia:
90%
●
Viento:
34
km/h
Jueves,
21
de
diciembre:
●
Temperatura
máxima:
17°C
●
Temperatura
mínima:
11°C
●
Probabilidad
de
lluvia:
13%
●
Viento:
9
km/h
Viernes,
22
de
diciembre:
●
Temperatura
máxima:
14°C
●
Temperatura
mínima:
7°C
●
Probabilidad
de
lluvia:
14%
●
Viento:
25
km/h
Sábado,
23
de
diciembre:
●
Temperatura
máxima:
15°C
●
Temperatura
mínima:
9°C
●
Probabilidad
de
lluvia:
70%
●
Viento:
10
km/h
Domingo,
24
de
diciembre:
●
Temperatura
máxima:
15°C
●
Temperatura
mínima:
9°C
●
Probabilidad
de
lluvia:
90%
●
Viento:
16
km/h


### Procesar el texto

In [11]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# limpiar y normalizar los textos
textos_limpios = [texto.strip() for texto in textos_pdf]
textos_minusculas = [texto.lower() for texto in textos_limpios]
textos_sin_caracteres_especiales = [re.sub(r'[^A-Za-z0-9\s]', '', texto) for texto in textos_minusculas]
textos_sin_stopwords = [' '.join([word for word in texto.split() if word not in ENGLISH_STOP_WORDS]) for texto in textos_sin_caracteres_especiales]
textos_tokenizados = [tokenizer.tokenize(texto) for texto in textos_sin_stopwords]

# previsualizar algunos textos 
for texto in textos_tokenizados[:5]:
    print(texto)


['cl', '##ima', 'en', 'co', '##ru', '##a', 'est', '##a', 'se', '##mana', 'lu', '##nes', '18', 'di', '##cie', '##mbre', 'temper', '##at', '##ura', 'mx', '##ima', '16', '##c', 'temper', '##at', '##ura', 'mn', '##ima', '10', '##c', 'pro', '##ba', '##bil', '##idad', 'll', '##u', '##via', '60', 'vie', '##nto', '13', 'km', '##h', 'mart', '##es', '19', 'di', '##cie', '##mbre', 'temper', '##at', '##ura', 'mx', '##ima', '16', '##c', 'temper', '##at', '##ura', 'mn', '##ima', '9', '##c', 'pro', '##ba', '##bil', '##idad', 'll', '##u', '##via', '90', 'vie', '##nto', '33', 'km', '##h', 'mir', '##col', '##es', '20', 'di', '##cie', '##mbre', 'temper', '##at', '##ura', 'mx', '##ima', '18', '##c', 'temper', '##at', '##ura', 'mn', '##ima', '12', '##c', 'pro', '##ba', '##bil', '##idad', 'll', '##u', '##via', '90', 'vie', '##nto', '34', 'km', '##h', 'ju', '##eves', '21', 'di', '##cie', '##mbre', 'temper', '##at', '##ura', 'mx', '##ima', '17', '##c', 'temper', '##at', '##ura', 'mn', '##ima', '11', '##c', 'p

### Crear Vector Store

In [16]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

vector_store = Chroma.from_texts(
    texts=textos_limpios, 
    collection_name="pdf_data", 
    embedding=embeddings, 
    persist_directory="./chroma_pdf_data"
)

# crear el retriever
recuperador = vector_store.as_retriever(search_kwargs={"k": 100})

### Implementar el LLM
Asegurarse que el docker de Ollama está funcionando con 'docker ps'

In [18]:
# definir la plantilla del prompt en español
plantilla = """Responde a la pregunta basándote únicamente en el siguiente contexto:
{context}

Pregunta: {question}
"""
prompt = ChatPromptTemplate.from_template(plantilla)

# seleccionar el modelo 
modelo_local = OllamaLLM(model="gemma:2b", base_url='http://localhost:11434')

# crear la cadena RAG
cadena = (
    {"context": recuperador, "question": RunnablePassthrough()}
    | prompt
    | modelo_local
    | StrOutputParser()
)

# hacer una pregunta de prueba 
respuesta = cadena.invoke("¿Cuál es la probabilidad de lluvia del lunes?")
print(respuesta)

Number of requested results 100 is greater than number of elements in index 3, updating n_results = 3


La probabilidad de lluvia del lunes en el contexto es del 60%.
